In [ ]:
import numpy as np
import pandas as pd
import os
import glob

from collections import defaultdict
from IPython.display import display, Latex

In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# API initialization


In [ ]:
pip install openai

In [ ]:
# API authentication parameters

my_key = ""
project_id = ""
org_id = ""

In [ ]:
from openai import OpenAI
from math import exp
import numpy as np
from IPython.display import display, HTML
import os

client = OpenAI(organization=org_id, project=project_id, api_key=my_key)

# logprobs POC

In [ ]:
# Taken from https://cookbook.openai.com/examples/using_logprobs

def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [ ]:
# Play with logprobs

OPENING = """Here is a question from the Data Structures course.
Solve the question.
Question:\n"""

QUESTION = """\textit{In a sequence of \(N\) Insert operations into a 2-3 tree (initially empty), let \(F(N)\) be the number of times a node split is performed in the tree when inserting \(N\) elements, in the worst case. Let \(G(N)\) be the total number of computer operations, when inserting \(N\) elements, in the worst case.}

\textit{What is the tightest upper bound on the ratio \(G(N)/F(N)\)?}

\begin{enumerate}
    \item \(\theta(1)\)
    \item \(\theta(\log(N))\)
    \item \(\theta(N)\)
    \item \(\theta(N \log(N))\)
    \item \(\theta\left(\frac{N}{\log(N)}\right)\)
\end{enumerate}
"""

api_response = get_completion(
    [{"role": "user", "content": OPENING + QUESTION}],
    model="gpt-4o-2024-08-06",
    logprobs=True,
    top_logprobs=5,
)
top_logprobs = api_response.choices[0].logprobs.content[0].top_logprobs
html_content = ""
for i, logprob in enumerate(top_logprobs, start=1):
    html_content += (
        f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
        f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
        f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
    )
display(HTML(html_content))



[ChatCompletionTokenLogprob(token='To', bytes=[84, 111], logprob=-0.09473735, top_logprobs=[TopLogprob(token='To', bytes=[84, 111], logprob=-0.09473735), TopLogprob(token='In', bytes=[73, 110], logprob=-2.5947373), TopLogprob(token='When', bytes=[87, 104, 101, 110], logprob=-4.3447375), TopLogprob(token='The', bytes=[84, 104, 101], logprob=-7.0947375), TopLogprob(token='A', bytes=[65], logprob=-7.0947375)]), ChatCompletionTokenLogprob(token=' solve', bytes=[32, 115, 111, 108, 118, 101], logprob=-0.3681069, top_logprobs=[TopLogprob(token=' solve', bytes=[32, 115, 111, 108, 118, 101], logprob=-0.3681069), TopLogprob(token=' determine', bytes=[32, 100, 101, 116, 101, 114, 109, 105, 110, 101], logprob=-1.8681068), TopLogprob(token=' address', bytes=[32, 97, 100, 100, 114, 101, 115, 115], logprob=-3.1181068), TopLogprob(token=' analyze', bytes=[32, 97, 110, 97, 108, 121, 122, 101], logprob=-3.2431068), TopLogprob(token=' find', bytes=[32, 102, 105, 110, 100], logprob=-3.4931068)]), ChatCo

# Multi-turn Conversation Example

In [ ]:
# Play with multi-turn conversation

OPENING = """Here is a multiple-choice question from the Data Structures course.
Solve the question.
Question:\n"""

QUESTION = """\noindent Consider a red-black tree as studied in class and described in Cormen (all Nils are considered leaves and their color is black). Let the height of a node \( y \) be defined as the length of the longest path (counted in nodes) upwards to \( y \). The count does not include \( y \) and the discussed nil. Let \( x \) be a node that is removed (note: the intention is the node removed and not the value removed) in the delete operation (as studied in class in a binary search tree). We are interested in the height of \( x \) before the removal. Then:
\begin{enumerate}
    \item The tightest upper bound on the height of \( x \) is 0.
    \item The tightest upper bound on the height of \( x \) is 1.
    \item The tightest upper bound on the height of \( x \) is 2.
    \item The tightest upper bound on the height of \( x \) is 3.
    \item There is no upper bound on the height of \( x \).
\end{enumerate}
"""

messages=[
  {
      "role": "user",
      "content": OPENING + QUESTION,
  }
]
chat_response1 = client.chat.completions.create(
  messages=messages,
  model="gpt-4o-2024-08-06"
)
print(chat_response1)
print("\n")

full_answer = chat_response1.choices[0].message.content
messages.extend(
    [{
      "role": "assistant",
      "content": full_answer,
    },
    {
      "role": "user",
      "content": "write the final answer as a number, without additional text.",
    }],
)
chat_response2 = client.chat.completions.create(
  messages=messages,
  model="gpt-4o-2024-08-06"
)
print(chat_response2)
print("\n")



ChatCompletion(id='chatcmpl-AKPh2lThtpqA3DIIWWR1Y2gnRcOoU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="In a red-black tree, the height of a node is defined as the length of the longest path upwards to that node, not including the node itself and considering the Nils (leaves), which are black. This question is asking about the height of a node \\(x\\) before it is removed during a delete operation. \n\nTo determine the upper bound on the height of \\(x\\), consider:\n\n1. The node \\(x\\) can be any node in the tree being deleted.\n2. The height of the node depends on its position within the tree.\n3. In the worst case, \\(x\\) could be a leaf node and those nodes have a height of 0 as there are no nodes above them.\n4. If \\(x\\) is an internal node, its height increases as it is positioned deeper in the tree.\n\nUsually, when considering a balanced binary search tree like a red-black tree, the deepest part (height) of a node is

# Run Analysis

### Create DF for results

In [ ]:
 # Create csv with the required columns
 cols = [
    "question_id",
    "model",
    "model_version",
    "question_prompt",
    "prompt_engineering_method",
    "timestamp",
    "answers_vector",
    "probabilities_vector",
    "tight_accuracy",
    "loose_accuracy"
]

df = pd.DataFrame(columns=cols)
df.to_csv("dataframe.tsv", sep="\t")

In [ ]:
df = pd.read_excel("combined_dataframes.xlsx")

## Process Questions

In [ ]:
GENERAL_FILENAME = "{model}-{model_version}_{question_id}_na_rep{rep_num}.txt"
COT_FILENAME = "{model}-{model_version}_{question_id}_cot_rep{rep_num}.txt"

FIRST_PROMPT = """Here is a multiple-choice/true-false question from the Data Structures course. Solve the question."""
SECOND_PROMPT = "write the final answer as a number, without additional text."
COT_SUFFIX = "Solve it step by step."

### Process Types C-D

In [1]:

def process_questions(df, model, model_version, num_reps):
  rows = []

  for idx in range(len(df)):
    year = df.loc[idx, "exam_year"]
    semester = df.loc[idx, "semester"]
    moed = df.loc[idx, "moed"]
    q_num = df.loc[idx , "question_number"]

    question_id = f"{year}{semester}{moed}_q{q_num}"
    question_text = df.loc[idx , "question_translation_latex"]
    dataset = df.loc[idx , "dataset"]
    has_sol = df.loc[idx , "has_solution"]
    q_type = df.loc[idx , "question_type"].upper()
    multiple_choice_answer = df.loc[idx, "multiple_choice_answer"]

    if dataset == "tested" and has_sol == True and q_type != "B":
      for rep in range(1, num_reps+1):
        row = {
            "question_id": question_id,
            "question_type": q_type,
            "multiple_choice_answer": multiple_choice_answer,
            "model": model,
            "model_version": model_version,
            "question_prompt": f"{FIRST_PROMPT}\nQuestion: {question_text}",
            "prompt_engineering_method": None,
            "rep_num": rep,
            "result_filename": GENERAL_FILENAME.format(model=model, model_version=model_version, question_id=question_id, rep_num=rep),
            "timestamp": None,
            "answers_vector": None,
            "probabilities_vector": None,
            "tight_accuracy": None,
            "loose_accuracy": None,
          }
        rows.append(row)

      for rep in range(1, num_reps+1):
        row = {
            "question_id": question_id,
            "question_type": q_type,
            "multiple_choice_answer": multiple_choice_answer,
            "model": model,
            "model_version": model_version,
            "question_prompt": f"{FIRST_PROMPT}\n{COT_SUFFIX}\nQuestion: {question_text}",
            "prompt_engineering_method": "COT",
            "rep_num": rep,
            "result_filename": COT_FILENAME.format(model=model, model_version=model_version, question_id=question_id, rep_num=rep),
            "timestamp": None,
            "answers_vector": None,
            "probabilities_vector": None,
            "tight_accuracy": None,
            "loose_accuracy": None
          }
        rows.append(row)

  df = pd.DataFrame(rows)
  return df


In [2]:
q_df = process_questions(df, "gpt", "4o-2024-08-06", 5)
q_df.to_excel("empty_dataframe.xlsx")
q_df

NameError: name 'df' is not defined

### Process Type B

In [ ]:
FIRST_PROMPT = """Here is a question from the Data Structures course. Solve the question."""
SECOND_PROMPT_FOR_TYPE_B = "Write the final answer, without additional text."
COT_SUFFIX = "Solve it step by step."

In [ ]:
def process_questions_type_b(df, model, model_version, num_reps):
  rows = []

  for idx in range(len(df)):
    year = df.loc[idx, "exam_year"]
    semester = df.loc[idx, "semester"]
    moed = df.loc[idx, "moed"]
    q_num = df.loc[idx , "question_number"]

    question_id = f"{year}{semester}{moed}_q{q_num}"
    question_text = df.loc[idx , "question_translation_latex"]
    dataset = df.loc[idx , "dataset"]
    has_sol = df.loc[idx , "has_solution"]
    q_type = df.loc[idx , "question_type"].upper()
    multiple_choice_answer = df.loc[idx, "multiple_choice_answer"]

    if dataset == "tested" and has_sol == True and q_type == "B":
      for rep in range(1, num_reps+1):
        row = {
            "question_id": question_id,
            "question_type": q_type,
            "multiple_choice_answer": multiple_choice_answer,
            "model": model,
            "model_version": model_version,
            "question_prompt": f"{FIRST_PROMPT}\nQuestion: {question_text}",
            "prompt_engineering_method": None,
            "rep_num": rep,
            "result_filename": GENERAL_FILENAME.format(model=model, model_version=model_version, question_id=question_id, rep_num=rep),
            "timestamp": None,
            "answers_vector": None,
            "probabilities_vector": None,
            "tight_accuracy": None,
            "loose_accuracy": None,
          }
        rows.append(row)

      for rep in range(1, num_reps+1):
        row = {
            "question_id": question_id,
            "question_type": q_type,
            "multiple_choice_answer": multiple_choice_answer,
            "model": model,
            "model_version": model_version,
            "question_prompt": f"{FIRST_PROMPT}\n{COT_SUFFIX}\nQuestion: {question_text}",
            "prompt_engineering_method": "COT",
            "rep_num": rep,
            "result_filename": COT_FILENAME.format(model=model, model_version=model_version, question_id=question_id, rep_num=rep),
            "timestamp": None,
            "answers_vector": None,
            "probabilities_vector": None,
            "tight_accuracy": None,
            "loose_accuracy": None
          }
        rows.append(row)

  df = pd.DataFrame(rows)
  return df


In [ ]:
q_df = process_questions_type_b(df, "gpt", "4o-2024-08-06", 5)
q_df.to_excel("empty_dataframe_type_b.xlsx")
q_df

,question_id,question_type,multiple_choice_answer,model,model_version,question_prompt,prompt_engineering_method,rep_num,result_filename,timestamp,answers_vector,probabilities_vector,tight_accuracy,loose_accuracy
0,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,1,gpt-4o-2024-08-06_2009ba_q12-a_na_rep1.txt,None,None,None,None,None
1,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,2,gpt-4o-2024-08-06_2009ba_q12-a_na_rep2.txt,None,None,None,None,None
2,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,3,gpt-4o-2024-08-06_2009ba_q12-a_na_rep3.txt,None,None,None,None,None
3,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,4,gpt-4o-2024-08-06_2009ba_q12-a_na_rep4.txt,None,None,None,None,None
4,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,5,gpt-4o-2024-08-06_2009ba_q12-a_na_rep5.txt,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,1,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep1.txt,None,None,None,None,None
296,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,2,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep2.txt,None,None,None,None,None
297,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,3,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep3.txt,None,None,None,None,None
298,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,4,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep4.txt,None,None,None,None,None


## Run Analysis: Type C-D Questions

In [ ]:
from pathlib import Path


In [ ]:
def run_analysis(df, directory, start=0):
  df_res = df.copy(deep=True)

  directory = Path(directory)
  directory.mkdir(exist_ok=True)

  try:
    for idx in range(start, len(df)):
      print(f"Processing {idx} - {df_res.loc[idx, 'question_id']}...")

      prompt = df_res.loc[idx, "question_prompt"]
      messages=[
        {
            "role": "user",
            "content": prompt,
        }
      ]
      first_chat_response = client.chat.completions.create(
        messages=messages,
        model="gpt-4o-2024-08-06",
      )
      df_res.loc[idx, "timestamp"] = first_chat_response.created
      full_answer = first_chat_response.choices[0].message.content

      # Write full answer
      filename = directory / df_res.loc[idx , "result_filename"]
      with open(filename, "w", encoding="utf-8") as file:
        file.write(first_chat_response.json())

      # Extract final answer
      messages.extend(
          [{
            "role": "assistant",
            "content": full_answer,
          },
          {
            "role": "user",
            "content": "write the final answer as a number, without additional text.",
          }],
      )
      second_chat_response = client.chat.completions.create(
        messages=messages,
        model="gpt-4o-2024-08-06"
      )
      short_answer = second_chat_response.choices[0].message.content
      df_res.loc[idx, "chat_response"] = short_answer

      # print("Prompt Number: " , idx)
      # print("Prompt: " , prompt)
      # print("Full Answer: " , full_answer)
      # print("Short Answer: " , short_answer)

      if idx % 10 == 0:
        df_res.to_excel("/content/drive/MyDrive/LLM Workshop/fixed-results.xlsx")

  except Exception as e:
    print(e)
  finally:
    df_res.to_excel("/content/drive/MyDrive/LLM Workshop/fixed-results.xlsx")
    return df_res


In [ ]:
run_analysis(q_df, directory="/content/drive/MyDrive/LLM Workshop/responses", start=0)

## Run Analysis: Type B Questions

In [ ]:
def run_analysis_on_type_b(df, directory, start=0):
  df_res = df.copy(deep=True)

  directory = Path(directory)
  directory.mkdir(exist_ok=True)

  try:
    for idx in range(start, len(df)):
      print(f"Processing {idx} - {df_res.loc[idx, 'question_id']}...")

      prompt = df_res.loc[idx, "question_prompt"]
      messages=[
        {
            "role": "user",
            "content": prompt,
        }
      ]
      first_chat_response = client.chat.completions.create(
        messages=messages,
        model="gpt-4o-2024-08-06",
      )
      df_res.loc[idx, "timestamp"] = first_chat_response.created
      full_answer = first_chat_response.choices[0].message.content

      # Write full answer
      filename = directory / df_res.loc[idx , "result_filename"]
      with open(filename, "w", encoding="utf-8") as file:
        file.write(first_chat_response.json())

      # Extract final answer
      messages.extend(
          [{
            "role": "assistant",
            "content": full_answer,
          },
          {
            "role": "user",
            "content": SECOND_PROMPT_FOR_TPYE_B,
          }],
      )
      second_chat_response = client.chat.completions.create(
        messages=messages,
        model="gpt-4o-2024-08-06"
      )
      short_answer = second_chat_response.choices[0].message.content
      df_res.loc[idx, "chat_response"] = short_answer

      # print("Prompt Number: " , idx)
      # print("Prompt: " , prompt)
      # print("Full Answer: " , full_answer)
      # print("Short Answer: " , short_answer)

      if idx % 10 == 0:
        df_res.to_excel("/content/drive/MyDrive/LLM Workshop/b-results.xlsx")

  except Exception as e:
    print(e)
  finally:
    df_res.to_excel("/content/drive/MyDrive/LLM Workshop/b-results.xlsx")
    return df_res

In [ ]:
q_df = pd.read_excel("b-results.xlsx")
first_na_index = q_df[q_df["chat_response"].isna()].index[0]
first_na_index

62

In [ ]:
run_analysis_on_type_b(q_df, directory="/content/drive/MyDrive/LLM Workshop/type-b-responses", start=0)

Processing 0 - 2009ba_q12-a...
Processing 1 - 2009ba_q12-a...
Processing 2 - 2009ba_q12-a...
Processing 3 - 2009ba_q12-a...
Processing 4 - 2009ba_q12-a...
Processing 5 - 2009ba_q12-a...
Processing 6 - 2009ba_q12-a...
Processing 7 - 2009ba_q12-a...
Processing 8 - 2009ba_q12-a...
Processing 9 - 2009ba_q12-a...
Processing 10 - 2010ba_q2-d...
Processing 11 - 2010ba_q2-d...
Processing 12 - 2010ba_q2-d...
Processing 13 - 2010ba_q2-d...
Processing 14 - 2010ba_q2-d...
Processing 15 - 2010ba_q2-d...
Processing 16 - 2010ba_q2-d...
Processing 17 - 2010ba_q2-d...
Processing 18 - 2010ba_q2-d...
Processing 19 - 2010ba_q2-d...
Processing 20 - 2010ba_q3-a...
Processing 21 - 2010ba_q3-a...
Processing 22 - 2010ba_q3-a...
Processing 23 - 2010ba_q3-a...
Processing 24 - 2010ba_q3-a...
Processing 25 - 2010ba_q3-a...
Processing 26 - 2010ba_q3-a...
Processing 27 - 2010ba_q3-a...
Processing 28 - 2010ba_q3-a...
Processing 29 - 2010ba_q3-a...
Processing 30 - 2010ba_q4-a...
Processing 31 - 2010ba_q4-a...
Processi

,question_id,question_type,multiple_choice_answer,model,model_version,question_prompt,prompt_engineering_method,rep_num,result_filename,timestamp,answers_vector,probabilities_vector,tight_accuracy,loose_accuracy,chat_response
0,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,1,gpt-4o-2024-08-06_2009ba_q12-a_na_rep1.txt,1730882290,None,None,None,None,"The array \([n, n-1, n-2, \ldots, 2, 1]\) cont..."
1,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,2,gpt-4o-2024-08-06_2009ba_q12-a_na_rep2.txt,1730882296,None,None,None,None,"Array: \([n, n-1, \ldots, 2, 1]\)\n\nNumber of..."
2,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,3,gpt-4o-2024-08-06_2009ba_q12-a_na_rep3.txt,1730882308,None,None,None,None,"The array \(\{n, n-1, \ldots, 2, 1\}\) contain..."
3,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,4,gpt-4o-2024-08-06_2009ba_q12-a_na_rep4.txt,1730882316,None,None,None,None,"The array \([n, n-1, \ldots, 1]\) contains the..."
4,2009ba_q12-a,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,None,5,gpt-4o-2024-08-06_2009ba_q12-a_na_rep5.txt,1730882323,None,None,None,None,The array sorted in descending order contains ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,1,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep1.txt,1730887062,None,None,None,None,"Yes, two AVL trees of size \( n \) each can be..."
296,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,2,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep2.txt,1730887076,None,None,None,None,"Yes, two AVL trees of size \( n \) each can be..."
297,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,3,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep3.txt,1730887088,None,None,None,None,"Yes, two AVL trees of size \( n \) each can be..."
298,2017AB_q1-c,B,na,gpt,4o-2024-08-06,Here is a question from the Data Structures co...,COT,4,gpt-4o-2024-08-06_2017AB_q1-c_cot_rep4.txt,1730887102,None,None,None,None,"Yes, two AVL trees of size \( n \) each can be..."


In [ ]:
!zip -r type-b-responses.zip /content/drive/MyDrive/LLM\ Workshop/type-b-*

  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/ (stored 0%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_2009ba_q12-a_na_rep1.txt (deflated 50%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_2009ba_q12-a_na_rep2.txt (deflated 53%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_2009ba_q12-a_na_rep3.txt (deflated 54%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_2009ba_q12-a_na_rep4.txt (deflated 55%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_2009ba_q12-a_na_rep5.txt (deflated 51%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_2009ba_q12-a_cot_rep1.txt (deflated 56%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_2009ba_q12-a_cot_rep2.txt (deflated 57%)
  adding: content/drive/MyDrive/LLM Workshop/type-b-responses/gpt-4o-2024-08-06_200

## Create log files

In [ ]:
from pathlib import Path
import json

In [ ]:
drive_path = Path("/content/drive/Othercomputers/My MacBook Pro/LLM workshop/final-project/part2/")

In [ ]:
def create_json_per_query(df):
  for idx in range(len(df)):
    query_dict = {}

    filename = df.loc[idx , "result_filename"]
    file_path = drive_path / "responses" / filename
    response_json = json.loads(file_path.read_text())

    query_dict["input_query"] = df.loc[idx , "question_prompt"]
    query_dict["model_full_answer"] = response_json
    query_dict["timestamp"] = response_json["created"]
    query_dict["model_short_answer"] = df.loc[idx , "chat_response"]
    query_dict["model_short_answer_after_manual_fixing"] = df.loc[idx , "chat_response_after_manual_fixing"].astype(str)

    print(query_dict)
    out_path = drive_path / "jsons" / (file_path.stem + ".json")
    print(out_path)
    with out_path.open(mode="w", encoding="utf-8") as json_file:
      json.dump(query_dict, json_file, indent=4)




In [ ]:
df = pd.read_excel("results.xlsx")
create_json_per_query(df)

{'input_query': 'Here is a multiple-choice/true-false question from the Data Structures course. Solve the question.\nQuestion: A sloppy minimum heap is a data structure similar to a heap, which instead of returning the minimum, returns one of the k smallest elements in the heap. Additionally, in the delete operation, it deletes the same element that it returned as the approximate minimum. The operations supported by the data structure (ADT) are:\n\\begin{itemize}\n    \\item \\texttt{Insert(x)} - insertion of a new element with key \\(x\\).\n    \\item \\texttt{FindApproxMin()} - return an approximate minimum (an element among the k smallest elements).\n    \\item \\texttt{DeleteApproxMin()} - deletion of the approximate minimum (deletion of the same element that if we performed \\texttt{FindApproxMin()} we would receive).\n\\end{itemize}\nFor \\(k=1\\) can a sloppy heap be implemented so that all operations in the ADT take amortized time \\(O(1)\\)?', 'model_full_answer': {'id': 'chat